# Optimizing model parameters

이제 우리는 학습할 모델과 데이터를 다 갖고 있다,  
우리의 데이터 위에서 파라미터들을 최적화하여 모델을 검증하고 테스트한다.  
모델을 학습하는 것은 iterative한 과정이다.  
매 epoch마다 output에 대한 예측을 생성하고,  
해당 예측의 error를 계산한다, 이것이 바로 loss이고,  
해당 각 파라미터들에 관한 error의 도함수를 수집하고,  
gradient descent(경사 하강법)을 사용해 파라미터들을 최적화한다.  

-> 더 자세히 이 과정을 알고 싶으면 다음 링크를 참고하라  
(홈페이지 가서 보삼)

## Prerequisite Code

앞 챕터에서 생성한 코드를 가져오자

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters

하이퍼파라미터들은 모델 최적화 과정을 제어하기 위해, 조절할 수 있는 파라미터들이다.  
다른 하이퍼파라미터 값들은 모델 학습과 convergence 비율에 영향을 줄 수 있다.  
(hyperparameter tuning을 참고할 것)

학습을 위한 다음과 같은 하이퍼파라미터들이 있다.

- Number of Epochs
  - 데이터셋 반복 수
- Batch size
  - 파라미터 업데이트 전 신경망으로 전파할 데이터 샘플들의 수
- Learning rate
  - 매 batch/epoch 마다 얼마나 모델 파라미터들을 업데이트 할 것인가.
  - 학습률

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop
우리의 하이퍼파라미터들을 설정하고,  
우리는 optimization loop와 함께 우리의 모델을 학습하고 최적화할 수 있다.  
optimization loop에서의 매 iteration을 epoch라고 부른다.

매 epoch는 2가지 주요 부분으로 구성되어 있다.

- The Train loop
  - 학습용 데이터셋으로 반복하고, 최적의 파라피터로 수렴하도록 시도한다.
- The Validation/Test Loop
  - 테스트용 데이터셋으로 반복하고, 모델 퍼포먼스 개선이 있었는지 체크한다
  
학습 루프에서 사용되는 개념들을 알아보자.

### Loss Function

아직 학습되지 않은 신경망은 틀린 답을 내기 쉽다.  
Loss function은 target value와 얻어진 결과의 다름의 정도를 측정하고,  
그것이 loss function이고 우리가 학습 동안 최소화 해야할 대상이다.

loss를 계산하기 위해 우리는 주어진 데이터 샘플을 사용해 prediction을 만들고  
그것을 true 데이터 라벨 값과 비교해야한다.

자주 쓰이는 loss functions는 다음과 같다.

- `nn.MSELoss`(Mean Square Error)
  - regression tasks(회귀)에 사용
- `nn.NLLLoss`(Negative Log Likelihood)
  - 분류 문제에 자주 사용
- `nn.CrossEntropyLoss`
  - `nn.LogSoftmax`와 `nn.NLLLoss`를 합쳐서 만든 것
  
우리는 `nn.CrossEntropyLoss`를 예제에서 사용한다.  
이를 통해 logits를 정규화하고, prediction error를 계산한다.


In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimizer

Optimization은 매 학습 단계에서 모델 에러를 줄이도록 모델 파라미터들을 적응시키는 과정이다.  
Optimization algorithms는 어떻게 이러한 과정들을 수행하는지 정의한다.  
(예를 들어 우리는 SGD, Stochastic Gradient Descent를 사용한다)

모든 optimization logic은 `optimizer` 객체로 캡슐화된다.  
여기서 우리는 SGD optimizer를 사용한다.

PyTorch에서 사용가능한 옵티마이저들은 다음과 같다.  
이들은 다양한 모델과 데이터의 경우에서 잘 작동한다.

- ADAM
- RMSProp

우리는 학습되어야 할 모델의 파라미터들을 등록하고,  
하이퍼파라미터인 학습률을 전달하여 옵티마이저를 초기화한다.

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Training loop안에서,  
3가지 단계가 수행된다.

- 모델 파라미터들의 기울기를 초기화하기 위해 `optimizer.zero_grad()`를 호출한다.
  - 기울기들은 기본적으로 더해진다.
  - 그래서 double-counting을 방지하기 위해, 우리는 명백히 매 iteration마다 이를 호출한다.
- `loss.backwards()`를 호출하면서 prediction loss를 역전파한다.
  - PyTorch는 loss 기울기를 매 파라미터마다 저장한다
- 우리의 기울기 값을 다 저장하고 난 후, 우리는 `optimizer.step()`을 호출한다
  - backward 단계에서 수집된 기울기 값들로 파라미터들을 업데이트한다.
  
(내 이해)
- `optimizer.zero_grad()`: 이전에 저장된 값 초기화
- `loss.backwards()`: 각 파라미터마다 loss 값으로 기울기 업데이트
- `optimizer.step()`: 업데이트된 기울기 값들로 파라미터 업데이트

## Full implementation

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

우리는 loss function과 optimizer을 초기화하고,  
이를 `train_loop`와 `test_loop`에 전달한다.

모델의 개선된 퍼포먼스를 추적하기 위한 epoch의 수를 늘리는 것을 편하게 생각해라.

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------
loss: 2.304460 [    0/60000]
loss: 2.292843 [ 6400/60000]
loss: 2.288857 [12800/60000]
loss: 2.286781 [19200/60000]
loss: 2.257131 [25600/60000]
loss: 2.251544 [32000/60000]
loss: 2.260139 [38400/60000]
loss: 2.237183 [44800/60000]
loss: 2.245186 [51200/60000]
loss: 2.213186 [57600/60000]
Test Error: 
 Accuracy: 36.6%, Avg loss: 0.034858 

Epoch 2
--------------------
loss: 2.235236 [    0/60000]
loss: 2.228448 [ 6400/60000]
loss: 2.208048 [12800/60000]
loss: 2.222889 [19200/60000]
loss: 2.141331 [25600/60000]
loss: 2.133320 [32000/60000]
loss: 2.167587 [38400/60000]
loss: 2.112585 [44800/60000]
loss: 2.153861 [51200/60000]
loss: 2.076620 [57600/60000]
Test Error: 
 Accuracy: 37.5%, Avg loss: 0.032915 

Epoch 3
--------------------
loss: 2.133714 [    0/60000]
loss: 2.127917 [ 6400/60000]
loss: 2.080720 [12800/60000]
loss: 2.120383 [19200/60000]
loss: 1.951496 [25600/60000]
loss: 1.950913 [32000/60000]
loss: 2.025215 [38400/60000]
loss: 1.926187 [44800/6000